# Energy Forecasting: Data Extraction

This notebook pulls data from the U.S. Energy Information Administration API

In [20]:
# Imports
import json
import os
import requests
import pandas as pd

from dotenv import load_dotenv
from time import sleep

In [2]:
# Load in environment variables
load_dotenv()

# Get API key
api_key = os.environ.get('API_KEY')

In [3]:
# Test API call with single day period
url = 'https://api.eia.gov/v2/electricity/rto/daily-fuel-type-data/data'
params = {'api_key': api_key}
header = {
    "frequency": "daily",
    "data": [
        "value"
    ],
    "facets": {
        "respondent": [
            "US48"
        ],
        "timezone": [
            "Pacific"
        ]
    },
    "start": "2019-01-01",
    "end": "2019-01-01",
    "sort": [
        {
            "column": "period",
            "direction": "desc"
        },
        {
            "column": "fueltype",
            "direction": "asc"
        }
    ],
    "offset": 0,
    "length": 5000
}

# Test call to the API
r = requests.get(url, params=params, headers={'X-Params': json.dumps(header)})

In [4]:
test_data = r.json()

test_data

{'response': {'total': 8,
  'dateFormat': 'YYYY-MM-DD',
  'frequency': 'daily',
  'data': [{'period': '2019-01-01',
    'respondent': 'US48',
    'respondent-name': 'United States Lower 48',
    'fueltype': 'COL',
    'type-name': 'Coal',
    'timezone': 'Pacific',
    'timezone-description': 'Pacific',
    'value': 2596861,
    'value-units': 'megawatthours'},
   {'period': '2019-01-01',
    'respondent': 'US48',
    'respondent-name': 'United States Lower 48',
    'fueltype': 'NG',
    'type-name': 'Natural gas',
    'timezone': 'Pacific',
    'timezone-description': 'Pacific',
    'value': 2799020,
    'value-units': 'megawatthours'},
   {'period': '2019-01-01',
    'respondent': 'US48',
    'respondent-name': 'United States Lower 48',
    'fueltype': 'NUC',
    'type-name': 'Nuclear',
    'timezone': 'Pacific',
    'timezone-description': 'Pacific',
    'value': 2476849,
    'value-units': 'megawatthours'},
   {'period': '2019-01-01',
    'respondent': 'US48',
    'respondent-name'

From the set up of the response, following factors are important:
* each fuel type has its own dictionary in data list
* there are 8 fuel types

Meaning the data list can be processed in chunks of 8 to make a single row that represents 1 date.

In [10]:
# Grab data for past years
url = f'https://api.eia.gov/v2/electricity/rto/daily-fuel-type-data/data'
params = {'api_key': api_key}

header = {
    "frequency": "daily",
    "data": [
        "value"
    ],
    "facets": {
        "respondent": [
            "US48"
        ],
        "timezone": [
            "Pacific"
        ]
    },
    "start": "2019-01-01",
    "end": "2023-12-31",
    "sort": [
        {
            "column": "period",
            "direction": "desc"
        },
        {
            "column": "fueltype",
            "direction": "asc"
        }
    ],
    "offset": 0,
    "length": 5000
}

In [11]:
# Function to break list into chunks
def make_chunks(data, size=8):
    
    return (data[i:i + size] for i in range(0, len(data), size))

In [21]:
# Make empty list to store row of each date as a dictionary
daily_data = []

# Call the API 3 times to get all data
for _ in range(3):
    r = requests.get(url, params=params, headers={'X-Params': json.dumps(header)})
    data = r.json()
    chunked_data = make_chunks(data['response']['data'])

    # Use each chunk to create dictionary for each date
    for chunk in chunked_data:
        row = {}
        row['date'] = chunk[0]['period']

        for i in range(8):
            row[f'{chunk[i]["type-name"].lower()}'] = chunk[i]['value']

        daily_data.append(row)
        
    # Increate the offset by 5000 in the header
    header['offset'] += 5000
    sleep(5)
    
print(len(daily_data))
daily_data[:5]

1826


[{'date': '2023-12-31',
  'coal': 1870097,
  'natural gas': 4270620,
  'nuclear': 2245684,
  'petroleum': 7293,
  'other': 221858,
  'solar': 305983,
  'hydro': 632266,
  'wind': 1074159},
 {'date': '2023-12-30',
  'coal': 1964710,
  'natural gas': 4369981,
  'nuclear': 2215492,
  'petroleum': 7661,
  'other': 224151,
  'solar': 322528,
  'hydro': 628762,
  'wind': 1064144},
 {'date': '2023-12-29',
  'coal': 1948705,
  'natural gas': 4482785,
  'nuclear': 2228321,
  'petroleum': 8311,
  'other': 216563,
  'solar': 312185,
  'hydro': 718066,
  'wind': 1109628},
 {'date': '2023-12-28',
  'coal': 1776641,
  'natural gas': 4480968,
  'nuclear': 2224448,
  'petroleum': 9639,
  'other': 209483,
  'solar': 309365,
  'hydro': 701614,
  'wind': 1119395},
 {'date': '2023-12-27',
  'coal': 1679634,
  'natural gas': 4397551,
  'nuclear': 2215389,
  'petroleum': 8236,
  'other': 218310,
  'solar': 262265,
  'hydro': 723729,
  'wind': 1093226}]

In [25]:
daily_data[-5:]

[{'date': '2019-01-05',
  'coal': 2482182,
  'natural gas': 3043584,
  'nuclear': 2442032,
  'petroleum': 5209,
  'other': 185661,
  'solar': 67261,
  'hydro': 824607,
  'wind': 702299},
 {'date': '2019-01-04',
  'coal': 2664270,
  'natural gas': 3543125,
  'nuclear': 2469281,
  'petroleum': 7251,
  'other': 193085,
  'solar': 110691,
  'hydro': 849861,
  'wind': 584117},
 {'date': '2019-01-03',
  'coal': 2702126,
  'natural gas': 3595766,
  'nuclear': 2485906,
  'petroleum': 8444,
  'other': 198361,
  'solar': 109039,
  'hydro': 869689,
  'wind': 709083},
 {'date': '2019-01-02',
  'coal': 2815323,
  'natural gas': 3652337,
  'nuclear': 2487248,
  'petroleum': 6718,
  'other': 197419,
  'solar': 105653,
  'hydro': 905440,
  'wind': 491233},
 {'date': '2019-01-01',
  'coal': 2596861,
  'natural gas': 2799020,
  'nuclear': 2476849,
  'petroleum': 10747,
  'other': 179678,
  'solar': 102279,
  'hydro': 804553,
  'wind': 561924}]

In [27]:
# Create dataframe with date as index
energy_df = pd.DataFrame.from_records(daily_data, index='date')

energy_df.sample(5)

,coal,natural gas,nuclear,petroleum,other,solar,hydro,wind
date,,,,,,,,
2023-08-27,2468940,5818632,2246692,10719,272375,519855,654691,497301
2020-11-18,2015267,3387001,2097198,32884,205236,176569,769592,1587320
2022-12-03,1954303,3784715,2214498,6149,183237,169855,590449,1241039
2019-05-14,2173874,3435066,2257327,13892,183951,233809,1058075,561831
2022-05-22,2135480,3725223,2134299,11977,185436,408364,775983,930723


In [28]:
# Change space to underscore in column name
energy_df = energy_df.rename({'natural gas': 'natural_gas'})

In [29]:
# Check for null values
energy_df.isna().sum()

coal           0
natural gas    0
nuclear        0
petroleum      0
other          0
solar          0
hydro          0
wind           0
dtype: int64

In [30]:
# Check for duplicates
energy_df.duplicated().sum()

0

In [32]:
# Save data to CSV
output_file = 'Data/energy_generation_data.csv'
energy_df.to_csv(output_file)